# Anomaly Detection Experiments
Using the original trained [GEE VAE](https://github.com/munhouiani/GEE)

## Environment Setting
Import libraries/packages/modules

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path

import psutil
import pyspark.sql.dataframe
from petastorm.etl.dataset_metadata import materialize_dataset
from petastorm.unischema import Unischema, dict_to_spark_row
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, unix_timestamp
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType

import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from torch import nn

import click
from petastorm import make_reader
from petastorm.pytorch import DataLoader
from pytorch_lightning import Trainer

import seaborn as sns
import shap
import matplotlib.pyplot as plt
from sklearn import metrics

## Download Data & Pretrained Model

In [ ]:
!pip install gdown
import gdown

In [ ]:
file_id="1ANCgjnnKaBiFwteeIdO0cF-_020jxfl2"
url = f'https://drive.google.com/uc?id={file_id}'
output = 'model_input.tar.gz'
gdown.download(url, output, quiet=False)

In [ ]:
!tar -xf model_input.tar.gz

In [ ]:
import gdown
file_id="118WX53iZQiTgM-qREd7EYhIH0SJaQzfX"
url = f'https://drive.google.com/uc?id={file_id}'
output = 'model.tar.gz'
gdown.download(url, output, quiet=False)

In [ ]:
!tar -xf model.tar.gz

## Utils

In [ ]:
feature_min_max = {
    'mean_duration': (0.0, 2042.86),
    'mean_packet': (1.0, 109214.27272727272),
    'mean_num_of_bytes': (28.0, 163795638.0909091),
    'mean_packet_rate': (0.0, 17224.14377310265),
    'mean_byte_rate': (0.0, 13902452.340182647),
    'std_duration': (0.0, 562.7625560888366),
    'std_packet': (0.0, 370614.95468242496),
    'std_num_of_bytes': (0.0, 543247494.7844237),
    'std_packet_rate': (0.0, 15783.66319664221),
    'std_byte_rate': (0.0, 16441139.793386225),
    'entropy_protocol': (0.0, 2.260220915066596),
    'entropy_dst_ip': (0.0, 13.787687869067254),
    'entropy_src_port': (0.0, 14.206227931544092),
    'entropy_dst_port': (0.0, 14.027301292191831),
    'entropy_flags': (0.0, 4.631615665225586)
}


def read_csv(spark: SparkSession, path: str) -> pyspark.sql.dataframe:
    """
    Read csv files as spark dataframe

    :param spark: spark session object
    :param path: path of dir containing csv files
    :type spark: SparkSession
    :type path: str
    :return: df
    :rtype: pyspark.sql.dataframe
    """

    # define csv schema
    schema = StructType([
        StructField('timestamp', StringType(), True),
        StructField('duration', DoubleType(), True),
        StructField('src_ip', StringType(), True),
        StructField('dst_ip', StringType(), True),
        StructField('src_port', LongType(), True),
        StructField('dst_port', LongType(), True),
        StructField('protocol', StringType(), True),
        StructField('flags', StringType(), True),
        StructField('forwarding_status', LongType(), True),
        StructField('type_of_service', LongType(), True),
        StructField('packet', LongType(), True),
        StructField('num_of_bytes', LongType(), True),
        StructField('label', StringType(), True),
    ])

    df = (
        spark
            .read
            .schema(schema)
            .csv(path)
    )

    # convert datetime column from string to unix_timestamp
    df = (
        df
            .withColumn('timestamp', unix_timestamp(col('timestamp'), 'yyyy-MM-dd HH:mm:ss'))
    )

    return df


def patch_time_windows(df: pyspark.sql.dataframe, window_seconds: int):
    """
    Generate time window by
    :param df: pyspark dataframe
    :param window_seconds: window size in second
    :type df: pyspark.sql.dataframe
    :type window_seconds: int
    :return: df
    :rtype: pyspark.sql.dataframe
    """
    time_window = from_unixtime(col('timestamp') - col('timestamp') % window_seconds)

    df = (
        df
            .withColumn('time_window', time_window)
    )

    return df


def init_local_spark():
    # initialise local spark
    os.environ['PYSPARK_PYTHON'] = sys.executable
    os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
    memory_gb = psutil.virtual_memory().available // 1024 // 1024 // 1024
    spark = (
        SparkSession
            .builder
            .master('local[*]')
            .config('spark.driver.memory', f'{memory_gb}g')
            .config('spark.driver.host', '127.0.0.1')
            .getOrCreate()
    )
    return spark


def normalise(x: float, min_val: float, max_val: float) -> float:
    norm_x = (x - min_val) / (max_val - min_val)
    if norm_x < 0:
        norm_x = 0.0
    elif norm_x > 1.0:
        norm_x = 1.0

    return norm_x


def row_generator(x):
    time_window, src_ip, feature, label = x
    return {
        'time_window': time_window,
        'src_ip': src_ip,
        'feature': np.expand_dims(np.array(feature, dtype=np.float32), axis=0),
        'label': label,
    }


def change_df_schema(spark: SparkSession, schema: Unischema, df: pyspark.sql.DataFrame) -> pyspark.sql.DataFrame:
    rows_rdd = (
        df
            .rdd
            .map(row_generator)
            .map(lambda x: dict_to_spark_row(schema, x))
    )

    df = spark.createDataFrame(
        rows_rdd,
        schema.as_spark_schema()
    )

    return df


def save_parquet_for_petastorm_parquet(spark: SparkSession, df: pyspark.sql.DataFrame, output_path: str,
                                       schema: Unischema):
    #output_path = Path(output_path).absolute().as_uri()
    output_path = 'file://' + str(Path(output_path).absolute())
    with materialize_dataset(spark, output_path, schema, row_group_size_mb=256):
        (
            df
                .write
                .mode('overwrite')
                .parquet(output_path)
        )

## Define Model
GEE VAE model

In [ ]:
class VAE(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def reparameterise(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterise(mu, logvar)
        return self.decoder(z), mu, logvar

    def loss_function(self, recon_x, x, mu, logvar):
        BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return BCE + KLD

    def training_step(self, batch, batch_idx):
        x = batch['feature']
        recon_x, mu, logvar = self(x)
        loss = self.loss_function(recon_x, x, mu, logvar)

        return {'loss': loss}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001, weight_decay=0.01)


class Encoder(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            # layer 1
            nn.Linear(
                in_features=69,
                out_features=512
            ),
            nn.ReLU(),
            # layer 2
            nn.Linear(
                in_features=512,
                out_features=512
            ),
            nn.ReLU(),
            # layer 3
            nn.Linear(
                in_features=512,
                out_features=1024
            ),
            nn.ReLU(),
        )

        # output
        self.mu = nn.Linear(
            in_features=1024,
            out_features=100
        )
        self.logvar = nn.Linear(
            in_features=1024,
            out_features=100
        )

    def forward(self, x):
        h = self.fc(x)
        return self.mu(h), self.logvar(h)


class Decoder(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            # layer 1
            nn.Linear(
                in_features=100,
                out_features=1024
            ),
            nn.ReLU(),
            # layer 2
            nn.Linear(
                in_features=1024,
                out_features=512
            ),
            nn.ReLU(),
            # layer 3
            nn.Linear(
                in_features=512,
                out_features=512
            ),
            nn.ReLU(),
            # output
            nn.Linear(
                in_features=512,
                out_features=69
            ),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.fc(x)

## Configuration

In [ ]:
anomaly_detector_model_path = 'model/vae.model'
data_path = 'model_input/test.model_input.parquet'

# get number of cores
num_cores = psutil.cpu_count(logical=True)
gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Load Trained GEE VAE Model

In [ ]:
anomaly_detector_model = VAE.load_from_checkpoint(checkpoint_path=anomaly_detector_model_path, map_location=torch.device(gpu))
anomaly_detector_model.eval()

## Run experiment

In [ ]:
def calc_recon_loss(recon_x, x, logvar = None, mu = None, loss_type: str = 'mse') -> list:
    """
    Return the reconstruction loss

    :param recon_x: reconstructed x, output from model
    :param x: original x
    :param logvar: variance, output from model, ignored when loss_type isn't 'bce+kd'
    :param mu: mean, output from model, ignored when loss_type isn't 'bce+kd'
    :param loss_type: method to compute loss, option: 'bce', 'mse', 'bce+kd'
    :return: list of reconstruct errors
    :rtype: list
    """

    loss_type = loss_type.lower()

    # 69 is the number of features
    if loss_type == 'mse':
        recon_error = F.mse_loss(recon_x, x, reduction='none').view(-1, 69).mean(dim=1)
    elif loss_type == 'bce':
        recon_error = F.binary_cross_entropy(recon_x, x, reduction='none').view(-1, 69).mean(dim=1)
    elif loss_type == 'bce+kd':
        bce = F.binary_cross_entropy(recon_x, x, reduction='none').view(-1, 69).mean(dim=1)
        kd = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        recon_error = bce + kd
    else:
        raise Exception('Invalid loss type: only support "mse", "bce", or "bce+kd"')

    return recon_error.tolist()

In [ ]:
%%time
reader = make_reader(
    'file://' + str(Path(data_path).absolute()), reader_pool_type='process', workers_count=num_cores,
    pyarrow_serialize=True, num_epochs=1
)
dataloader = DataLoader(reader, batch_size=300, shuffling_queue_capacity=4096)

labels = []
xes = []
mse = []

for data in dataloader:
    data_x = data['feature']
    data_labels = data['label']
    xes.extend(np.squeeze(data_x.numpy()))
    labels.extend(data_labels)
    reconstruction, mu, logvar = anomaly_detector_model(data_x)
    mse.extend(calc_recon_loss(reconstruction, data_x, loss_type='mse'))

features = []
for item in xes:
    features.append(np.squeeze(item).astype(float))
test_data = pd.DataFrame(
    {
        'labels': labels,
        'features': features,
        'mse': mse
    }
)
test_data.to_feather('results_ad_test.feather.with_mse')